In [1]:
# pip install mysql-connector-python

# To create MySQL tables using Python, 
# you typically use one of the two libraries: 
# mysql-connector-python or SQLAlchemy.
from sqlalchemy import create_engine
import mysql.connector

In [2]:
# Connect to MySQL server
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="0000"
)

# Create a cursor object
cursor = connection.cursor()

# SQL query to create a new database
cursor.execute("CREATE DATABASE IF NOT EXISTS northwind_star_schema")


In [3]:
# Switch to the newly created database
cursor.execute("USE northwind_star_schema")

# SQL query to create the customer_dim table
create_customer_dim = """
CREATE TABLE IF NOT EXISTS customer_dim (
    customer_key INT AUTO_INCREMENT PRIMARY KEY,
    customer_id CHAR(40) NOT NULL,
    company_name VARCHAR(40) NOT NULL,
    contact_name VARCHAR(30),
    contact_title VARCHAR(30)
);
"""
cursor.execute(create_customer_dim)

# SQL query to create the date_dim table
create_date_dim = """
CREATE TABLE IF NOT EXISTS date_dim (
    date_key INT NOT NULL PRIMARY KEY,
    full_date DATE,
    year INT,
    quarter INT,
    quarter_name VARCHAR(20),
    month INT,
    month_name VARCHAR(20),
    week INT,
    day INT,
    day_name VARCHAR(9)
);
"""
cursor.execute(create_date_dim)

# SQL query to create the employee_dim table
create_employee_dim = """
CREATE TABLE IF NOT EXISTS employee_dim (
    employee_key INT AUTO_INCREMENT PRIMARY KEY,
    employee_id INT NOT NULL,
    first_name VARCHAR(10) NOT NULL,
    last_name VARCHAR(20) NOT NULL,
    title VARCHAR(30),
    title_of_courtesy VARCHAR(25),
    birth_date DATETIME,
    hire_date DATETIME,
    address VARCHAR(60),
    city VARCHAR(15),
    region VARCHAR(15),
    country VARCHAR(15),
    postal_code VARCHAR(10),
    reports_to SMALLINT
);
"""
cursor.execute(create_employee_dim)

# SQL query to create the product_dim table
create_product_dim = """
CREATE TABLE IF NOT EXISTS product_dim (
    product_key INT AUTO_INCREMENT PRIMARY KEY,
    product_id INT NOT NULL,
    product_name VARCHAR(40) NOT NULL,
    quantity_per_unit VARCHAR(20),
    unit_price REAL,
    units_in_stock SMALLINT,
    units_on_order SMALLINT,
    reorder_level SMALLINT,
    discontinued SMALLINT,
    category_name VARCHAR(45) NOT NULL
);
"""
cursor.execute(create_product_dim)

# SQL query to create the supplier_dim table
create_supplier_dim = """
CREATE TABLE IF NOT EXISTS supplier_dim (
    supplier_key INT AUTO_INCREMENT PRIMARY KEY,
    supplier_id INT NOT NULL,
    company_name VARCHAR(40) NOT NULL,
    contact_title VARCHAR(30),
    address VARCHAR(60),
    city VARCHAR(15),
    region VARCHAR(15),
    country VARCHAR(15),
    postal_code VARCHAR(10)
);
"""
cursor.execute(create_supplier_dim)

# SQL query to create the shipper_dim table
create_shipper_dim = """
CREATE TABLE IF NOT EXISTS shipper_dim (
    shipper_key INT AUTO_INCREMENT PRIMARY KEY,
    shipper_id INT NOT NULL,
    company_name VARCHAR(40) NOT NULL
);
"""
cursor.execute(create_shipper_dim)

# SQL query to create the ship_info table
create_ship_info = """
CREATE TABLE IF NOT EXISTS ship_info (
    shipinfo_key INT AUTO_INCREMENT PRIMARY KEY,
    customer_key INT,
    ship_address VARCHAR(60),
    ship_city VARCHAR(15),
    ship_region VARCHAR(15),
    ship_country VARCHAR(15),
    FOREIGN KEY (customer_key) REFERENCES customer_dim(customer_key)
);
"""
cursor.execute(create_ship_info)

# SQL query to create the orders_fact table
create_orders_fact = """
CREATE TABLE IF NOT EXISTS orders_fact (
    order_key INT AUTO_INCREMENT PRIMARY KEY,
    orderID INT NOT NULL,
    employee_key INT,
    shipper_key INT,
    customer_key INT,
    supplier_key INT,
    product_key INT,
    shipinfo_key INT,
    order_date INT,
    required_date INT,
    shipped_date INT,
    freight REAL,
    unit_price REAL,
    quantity INT,
    discount REAL,
    actual_cost REAL,
    FOREIGN KEY (employee_key) REFERENCES employee_dim(employee_key),
    FOREIGN KEY (shipper_key) REFERENCES shipper_dim(shipper_key),
    FOREIGN KEY (customer_key) REFERENCES customer_dim(customer_key),
    FOREIGN KEY (supplier_key) REFERENCES supplier_dim(supplier_key),
    FOREIGN KEY (product_key) REFERENCES product_dim(product_key),
    FOREIGN KEY (shipinfo_key) REFERENCES ship_info(shipinfo_key),
    FOREIGN KEY (order_date) REFERENCES date_dim(date_key),
    FOREIGN KEY (required_date) REFERENCES date_dim(date_key),
    FOREIGN KEY (shipped_date) REFERENCES date_dim(date_key)
);
"""
cursor.execute(create_orders_fact)

# Populate date_dim table
start_date = "1996-07-04"
end_date = "2030-12-31"

query = """
INSERT INTO date_dim (date_key, full_date, year, quarter, quarter_name, month, month_name, week, day, day_name)
VALUES (%s, %s, YEAR(%s), QUARTER(%s), CONCAT('Q', QUARTER(%s)), MONTH(%s), MONTHNAME(%s), WEEK(%s), DAY(%s), DAYNAME(%s));
"""

# Python loop to populate the date_dim table
from datetime import datetime, timedelta

date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date, date_format)
end_date = datetime.strptime(end_date, date_format)
current_date = start_date

while current_date <= end_date:
    date_key = int(current_date.strftime("%Y%m%d"))
    data = (date_key, current_date, current_date, current_date, current_date, current_date, current_date, current_date, current_date, current_date)
    cursor.execute(query, data)
    current_date += timedelta(days=1)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Tables and data created successfully!")


IntegrityError: 1062 (23000): Duplicate entry '19960704' for key 'date_dim.PRIMARY'